# Titanic Challenge

In [1]:
import numpy as np
# !pip install plotly
import plotly.express as px

In [2]:
import pandas as pd

train = pd.read_csv('/kaggle/input/d/lonesomebucket/titanic/train.csv')
train.head()

,Id,Outcome,Pclass,Name,Sex,Age,Sibsp,Parch,Fare
0,1,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,2,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,3,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,4,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,5,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


Now let's bin and onehot

In [3]:
agebins = pd.cut(train.Age,bins=[0,6,13,19,60,999],labels=['infant','child','teen','adult','senior'])
train['agebins'] = agebins
train = pd.get_dummies(train,columns=['Sex','Pclass','agebins'],drop_first=True)
train['not_alone'] = (train['Sibsp'] + train['Parch']).apply(lambda x: 1 if x > 0 else 0)   
train['LogFare'] = np.log(train['Fare'].replace(0, 0.01).replace(np.NaN, 0.01))
train.head()

,Id,Outcome,Name,Age,Sibsp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,agebins_child,agebins_teen,agebins_adult,agebins_senior,not_alone,LogFare
0,1,0,Mr. Owen Harris Braund,22.0,1,0,7.2500,True,False,True,False,False,True,False,1,1.981001
1,2,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,38.0,1,0,71.2833,False,False,False,False,False,True,False,1,4.266662
2,3,1,Miss. Laina Heikkinen,26.0,0,0,7.9250,False,False,True,False,False,True,False,0,2.070022
3,4,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,35.0,1,0,53.1000,False,False,False,False,False,True,False,1,3.972177
4,5,0,Mr. William Henry Allen,35.0,0,0,8.0500,True,False,True,False,False,True,False,0,2.085672


In [5]:
train["Sex_male"] = train["Sex_male"].astype(int)
train["Pclass_2"] = train["Pclass_2"].astype(int)
train["Pclass_3"] = train["Pclass_3"].astype(int)
train["agebins_child"] = train["agebins_child"].astype(int)
train["agebins_teen"] = train["agebins_teen"].astype(int)
train["agebins_adult"] = train["agebins_adult"].astype(int)
train["agebins_senior"] = train["agebins_senior"].astype(int)
train.head()

,Id,Outcome,Name,Age,Sibsp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,agebins_child,agebins_teen,agebins_adult,agebins_senior,not_alone,LogFare
0,1,0,Mr. Owen Harris Braund,22.0,1,0,7.2500,1,0,1,0,0,1,0,1,1.981001
1,2,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,38.0,1,0,71.2833,0,0,0,0,0,1,0,1,4.266662
2,3,1,Miss. Laina Heikkinen,26.0,0,0,7.9250,0,0,1,0,0,1,0,0,2.070022
3,4,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,35.0,1,0,53.1000,0,0,0,0,0,1,0,1,3.972177
4,5,0,Mr. William Henry Allen,35.0,0,0,8.0500,1,0,1,0,0,1,0,0,2.085672


Now let's start getting ready to model

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train[["Id", "Sex_male",
                                                            "Pclass_2", "Pclass_3",
                                                            "agebins_child",
                                                            "agebins_teen","agebins_adult",
                                                            "agebins_senior",
                                                            "LogFare", "not_alone"]],
                                                    train['Outcome'], test_size=0.2, random_state=22)

In [8]:
X_train.shape
X_train.head()

,Id,Sex_male,Pclass_2,Pclass_3,agebins_child,agebins_teen,agebins_adult,agebins_senior,LogFare,not_alone
440,441,0,1,0,0,0,1,0,2.564949,0
194,195,0,0,0,0,0,1,0,4.987167,0
215,216,0,0,1,0,0,1,0,2.070022,0
2,3,0,0,1,0,0,1,0,2.070022,0
75,76,1,0,1,0,0,1,0,2.066331,0


In [9]:
X_train.not_alone.describe()

count    572.000000
mean       0.396853
std        0.489673
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: not_alone, dtype: float64

In [13]:
import tensorflow as tf
x = np.array(train[['Sex_male','Pclass_2','Pclass_3',
                     'agebins_child','agebins_teen','agebins_adult','agebins_senior',
                     'LogFare','not_alone']])
y = np.array(train['Outcome'])
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3,activation='sigmoid'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x,y,epochs=20)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3992 - loss: 0.7281
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3888 - loss: 0.7168  
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4413 - loss: 0.7039 
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4926 - loss: 0.6934 
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6325 - loss: 0.6827 
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6212 - loss: 0.6775 
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6179 - loss: 0.6712 
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6145 - loss: 0.6666 
Epoch 9/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5884 - loss: 0.6664 
Epoch 10/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5849 - loss: 0.6671 
Epoch 11/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5970 - loss: 0.6583 
Epoch 12/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 